In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from google_play_scraper import app, Sort, reviews_all, search
from app_store_scraper import AppStore
# from itunes_app_scraper.scraper import AppStoreScraper
import pyodbc as odbc
import sys
import openai
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession
import openpyxl

# Load Support Data

In [ ]:
storage_account_name = "storage_account_name"
container_name = "container_name"
mount_point = "/mnt/mount_point"
storage_account_key = "storage_account_key"

# Check if already mounted
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    print(f"{mount_point} is already mounted.")
else:
    # Mount the container
    dbutils.fs.mount(
        source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
        mount_point=mount_point,
        extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
    )
    print(f"Mounted {mount_point} successfully.")

# Correct file path with /dbfs prefix
file_path = "/dbfs/mnt/file_path.xlsx"

try:
    # Read the Excel file into a Pandas DataFrame
    Apps_file = pd.read_excel(file_path, sheet_name='sheet_name')
    print("appsprodversion file loaded successfully!")
except Exception as e:
    print(f"Error loading appsprodversion file: {e}")

# Optionally unmount (if you want a clean workspace):
dbutils.fs.unmount(mount_point)

Mounted /mnt/appsprodversion successfully.
appsprodversion file loaded successfully!
/mnt/appsprodversion has been unmounted.


True

In [ ]:
# Use support file data
Apps_file.fillna("", inplace=True)

countries = Apps_file['Tenant'].str[:2].to_list()
countries = [x.lower() for x in countries]

tenant = list(Apps_file['Tenant'])

iOS_apps_names = []
iOS_apps_ids = []
for i in range(0, Apps_file.shape[0]):
    link = Apps_file['Link Store iOS'][i]
    if not "N/A" in link:
        id_start = link.find("/id") + 3  # Index after "/id"
        id_end = link.find("?", id_start)  # Find index of "?" after the ID
        if id_end != -1:
            # If "?" is found after the ID, extract the ID before "?"
            app_id = link[id_start:id_end]
        else:
            # If "?" is not found, extract the ID till the end of the string
            app_id = link[id_start:]
        iOS_apps_names.append(link[link.find("app/") + 4:link.find("/id")])
        iOS_apps_ids.append(app_id)
    else:
        iOS_apps_names.append("")
        iOS_apps_ids.append("")

Android_apps_names = []
for i in range(0, Apps_file.shape[0]):
    Android_apps_names.append(Apps_file['Link Store Android'][i][Apps_file['Link Store Android'][i].find("?id=") + 4 : ])

In [ ]:
# Load base data
dw_jdbcHostname = "server"
dw_jdbcPort = 1433
dw_jdbcDatabase = "database"
dw_aadUsername = "username"  # Your Azure AD user
dw_aadPassword = "password"  # Your Azure AD password

# JDBC URL format for SQL Server with Azure AD authentication
dw_jdbcUrl = f"jdbc:sqlserver://{dw_jdbcHostname}:{dw_jdbcPort};database={dw_jdbcDatabase};user={dw_aadUsername};password={dw_aadPassword}"

In [ ]:
# Reviews base file query
query_1 = """SELECT * FROM [dbo].[ReviewsApps]"""

# Load data using Spark JDBC for Reviews
reviews_base_file = spark.read.format("jdbc") \
    .option("url", dw_jdbcUrl) \
    .option("query", query_1) \
    .option("user", dw_aadUsername) \
    .option("password", dw_aadPassword) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

reviews_base_file = reviews_base_file.toPandas()

# Ratings base file query
query_2 = """SELECT * FROM [dbo].[RatingsApps]"""

# Load data using Spark JDBC for Ratings
ratings_base_file = spark.read.format("jdbc") \
    .option("url", dw_jdbcUrl) \
    .option("query", query_2) \
    .option("user", dw_aadUsername) \
    .option("password", dw_aadPassword) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

ratings_base_file = ratings_base_file.toPandas()

In [ ]:
# Update Reviews DB
def convert_to_tuple(*args):
    for x in args:
        if not isinstance(x, list) and not isinstance(x, tuple):
            return []
    size = float("inf")
    for x in args:
        size = min(size, len(x))
    result = []
    for i in range(size):
        result.append(tuple([x[i] for x in args]))
    return result

# Reviews

In [ ]:
# iOS
iOS_reviews = pd.DataFrame()
for i, app in enumerate(iOS_apps_names):   
    try:
        if app != '':
            info = AppStore(country = countries[i], 
                            app_name = app,
                            app_id = iOS_apps_ids[i])
            
            # Send a GET request to the URL
            response = requests.get(info.url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find all elements containing the word "userName" in the text within review_elements
                review_elements = soup.find_all(lambda tag: tag and "userName" in tag.text)

                # If there are review elements found, proceed to fetch reviews using your function
                if review_elements:                    
                    info.review()   # how_many parameter setea cuantas reviews             
                    
                    reviews = pd.DataFrame(info.reviews)
                    
                    reviews['Tenant'] = tenant[i]
                    reviews['reviewId'] = reviews['userName'].astype(str) + "-" + reviews['date'].astype(str)

                    iOS_reviews = pd.concat([iOS_reviews, reviews])

        else:
            pass    
    except:
        pass

iOS_reviews['channel'] = 'iOS'
iOS_reviews['language'] = 'en'

In [ ]:
# ANDROID
reviews_esp = pd.DataFrame()
not_found_esp = 0
 
for i, app in enumerate(Android_apps_names):
    try:    
        reviews = pd.DataFrame(reviews_all(
            app,
            sleep_milliseconds = 0, # defaults to 0
            lang = 'es', # defaults to 'en'
            country = countries[i], # defaults to 'us'
            sort = Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
        ))
        reviews['Tenant'] = tenant[i]
        
        reviews_esp = pd.concat([reviews_esp, reviews])

    except:
        not_found_esp = not_found_esp + 1
        not_found_esp_Tenant = tenant[i]
        pass

reviews_esp['language'] = 'esp'


reviews_eng = pd.DataFrame()
not_found_eng = 0

for i, app in enumerate(Android_apps_names):
    try:    
        reviews = pd.DataFrame(reviews_all(
            app,
            sleep_milliseconds = 0, # defaults to 0
            lang = 'en', # defaults to 'en'
            country = countries[i], # defaults to 'us'
            sort = Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
        ))
        reviews['Tenant'] = tenant[i]
        
        reviews_eng = pd.concat([reviews_eng, reviews])
        
    except:
        not_found_eng = not_found_eng + 1
        not_found_eng_Tenant = tenant[i]
        pass 

reviews_eng['language'] = 'en'

# Concat both datasets
Android_reviews = pd.concat([reviews_esp, reviews_eng], ignore_index=True) 

# Channel column
Android_reviews['channel'] = 'Android'
Android_reviews.rename(columns={'at':'date', 'content':'review', 'score':'rating'}, inplace=True)
Android_reviews = Android_reviews[['reviewId','date','review','rating','Tenant','channel','language']]

Android_reviews.reset_index(inplace=True, drop=True)

In [ ]:
# Combine datasets
reviews_total = pd.concat([iOS_reviews, Android_reviews])

reviews_total = reviews_total[['reviewId','date','review','rating','Tenant','channel','language']]

reviews_total.reset_index(drop=True, inplace=True)

## Sentiment Analysis

In [ ]:
rows_to_delete = []
for i, reviewId in enumerate(reviews_total['reviewId']):
    if reviews_base_file['reviewId'].eq(str(reviewId)).any():
       rows_to_delete.append(i) 
    else:
        pass
df_new_reviews = reviews_total.drop(rows_to_delete)
df_new_reviews['sentiment'] = None

df_new_reviews.reset_index(drop=True, inplace=True)

In [ ]:
# openai==0.28
openai.api_key = "api_key"

# Define a function to classify a review
def classify_review(review):
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Classify the sentiment in this review in 'Positive' and 'Negative'. Just one of this two categories and nothing else:\n \"{review}\"",
        temperature=0,
        max_tokens=60,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    # Extract the sentiment from the response
    sentiment = response.choices[0].text.strip()
    return sentiment

# Apply the function to the reviews column
df_new_reviews['sentiment'] = df_new_reviews['review'].apply(classify_review)

df_new_reviews.fillna('', inplace=True)

# Trim review length to be max 500
df_new_reviews['review'] = df_new_reviews['review'].apply(lambda x: x[:500] if isinstance(x, str) and len(x) > 500 else x)

## Upload Reviews

In [ ]:
if len(df_new_reviews) > 1:
    # Create a Spark dataframe
    spark_df_new_reviews = spark.createDataFrame(df_new_reviews)

    # Insert data into database
    spark_df_new_reviews.write.format("jdbc").mode("append").option("url", dw_jdbcUrl).option("dbtable", "dbo.ReviewsApps").option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").save()
else:
    print("df_new_reviews is empty, not creating a Spark DataFrame.")

# App Ratings

In [ ]:
# Ratings Google Play
from google_play_scraper import app

Android_appsinfo = pd.DataFrame()

for i, app_id in enumerate(Android_apps_names):  
    if ".com" in app_id: 
        try:
            android_ratings_info = app(
                        app_id,
                        lang = 'es', # defaults to 'en'
                        country = countries[i], # defaults to 'us'
                    )

            android_keys = ['realInstalls', 'score', 'ratings']

            android_filtered_dict = {key: value for key, value in android_ratings_info.items() if key in android_keys} 
            
            android_ratings_info = pd.DataFrame(android_filtered_dict, index=[i])
            android_ratings_info['Tenant'] = tenant[i]

            Android_appsinfo = pd.concat([Android_appsinfo, android_ratings_info])
        except:
            pass    
    else:
        pass

Android_appsinfo['channel'] = 'Android'
Android_appsinfo = Android_appsinfo[['Tenant', 'score', 'ratings', 'realInstalls']]
Android_appsinfo.rename(columns={'realInstalls': 'Android_installs', 'score': 'Android_app_score', 'ratings':'Android_ratings'}, inplace=True)

In [ ]:
# Ratings iOS
iOS_appsinfo = pd.DataFrame()

for i, app in enumerate(iOS_apps_names):   
    if app != '':
        try:
            endpoint = "https://itunes.apple.com/lookup?id="
            response = requests.get(endpoint + iOS_apps_ids[i])
            data = json.loads(response.text)
            iOS_ratings_info = data["results"][0]

            iOS_keys = ['averageUserRating', 'userRatingCount']
            
            iOS_filtered_dict = {key: value for key, value in iOS_ratings_info.items() if key in iOS_keys}

            iOS_app_info = pd.DataFrame(iOS_filtered_dict, index=[i])
            iOS_app_info['Tenant'] = tenant[i]
            
            iOS_appsinfo = pd.concat([iOS_appsinfo, iOS_app_info])
        except:
            pass    
    else:
        pass

iOS_appsinfo['channel'] = 'iOS'
iOS_appsinfo = iOS_appsinfo[['Tenant', 'averageUserRating', 'userRatingCount']]
iOS_appsinfo.rename(columns={'averageUserRating': 'iOS_app_score', 'userRatingCount':'iOS_ratings'}, inplace=True)

In [ ]:
# Combined Ratings
combined_ratings = Android_appsinfo.merge(iOS_appsinfo, on='Tenant')

# Fill values with NaN for transformatio
combined_ratings = combined_ratings.fillna(np.nan)

combined_ratings['Android_app_score'] = round(combined_ratings['Android_app_score'], 2)
combined_ratings['iOS_app_score'] = round(combined_ratings['iOS_app_score'], 2)

combined_ratings['date'] = pd.to_datetime('today').normalize()

In [ ]:
# Just keep new values

# Ensure both DataFrames are in pandas format
ratings_base_file_pd = ratings_base_file.copy()
combined_ratings_pd = combined_ratings.copy()

# Create a set of tuples (Tenant, date) from the existing ratings in the database
existing_records = set(
    zip(ratings_base_file_pd['Tenant'], ratings_base_file_pd['date'])
)

# Filter the combined_ratings DataFrame to include only new records
df_new_ratings = combined_ratings_pd[
    ~combined_ratings_pd.apply(lambda row: (row['Tenant'], row['date']), axis=1).isin(existing_records)
]

# Convert to float
df_new_ratings['Android_app_score'] = df_new_ratings['Android_app_score'].astype(float).fillna(0)
df_new_ratings['iOS_app_score'] = df_new_ratings['iOS_app_score'].astype(float).fillna(0)

# Log the results
if df_new_ratings.empty:
    pass
else:
    print(f"New ratings to add: {len(df_new_ratings)}")

New ratings to add: 64


## Upload Ratings

In [ ]:
if len(df_new_ratings) > 1:
    # Create a Spark dataframe
    spark_df_new_ratings = spark.createDataFrame(df_new_ratings)

    # Insert data into database
    spark_df_new_ratings.write.format("jdbc").mode("append").option("url", dw_jdbcUrl).option("dbtable", "dbo.RatingsApps").option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").save()
else:
    print("df_new_ratings is empty, not creating a Spark DataFrame.")